# EMG + Kraft CTF: Python + NumPy (DA)

Velkommen! Denne CTF guider dig gennem at læse et laboratoriedatasæt (EMG og kraft), parse det med **NumPy** (uden pandas) og plotte med **matplotlib**. Til sidst laver du basal EMG-efterbehandling.

**Regler**
- Brug **NumPy** til parsing og signalbehandling og **matplotlib** til plots.
- Ingen pandas/seaborn, medmindre det udtrykkeligt er tilladt.
- Hver opgave har et **FLAG-format**: `CTF{...}`. Beregn de krævede værdier og skriv flagget *præcis* som angivet.

**Data**
- Fil: `scope_11.csv`
- Hint: filen har en headerlinje med akser/enheder, og tal er skrevet i videnskabelig notation som `+10.0000E-03`.

God arbejdslyst 🍀

## Opsætning
Importer de tilladte biblioteker og angiv stien til CSV-filen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

PATH = 'scope_11.csv'  # læg filen ved siden af denne notebook


## Udfordring 1 — Læs & parse
**Mål:** Læs CSV **uden pandas** og parse tre kolonner til arrays: tid (sekunder), kanal1 (Volt), kanal2 (Volt). Første linje er labels/enheder. Værdier kan se ud som `+10.0592000E+00`.

**Tips:**
- Brug `np.genfromtxt` (dtype=str), rens tekst og konverter til float.
- Fjern evt. foranstillet `+`, bevar `E`-notationen.
- Skip headerlinjen.

**FLAG 1 (størrelse):** `CTF{N_samples}` hvor `N_samples` er antal **datarækker** (uden header).

In [ ]:
raw = np.genfromtxt(PATH, delimiter=',', dtype=str)
data = raw[1:]  # skip header

def parse_sci(str_array):
    s = np.char.strip(str_array)
    s = np.char.replace(s, '+', '')
    s = np.char.replace(s, ',', '.')
    return s.astype(float)

t = parse_sci(data[:, 0])
ch1 = parse_sci(data[:, 1])
ch2 = parse_sci(data[:, 2])

N_samples = t.size
print('N_samples =', N_samples)
# FLAG 1: CTF{...}

## Udfordring 2 — Samplingsfrekvens
**Mål:** Beregn samplingsfrekvensen `fs` i Hz ud fra efterfølgende tidsstempler.

**FLAG 2:** `CTF{fs_in_Hz}` (du må gerne runde til heltal, hvis passende).

In [ ]:
dt = np.diff(t)
fs = 1.0 / np.mean(dt)
print('fs ≈', fs)
# FLAG 2: CTF{...}

## Udfordring 3 — Plot & kanalidentifikation
**Mål:** Plot begge kanaler over tid (to separate figurer). Identificér hvilken kanal der er **EMG** (højfrekvent, bursty) og hvilken der er **kraft** (langsomt varierende).

**Tilladt:** Kun `matplotlib`. Ingen styles eller farver.

**FLAG 3:** `CTF{EMG=1,FORCE=2}` hvis kanal 1 er EMG og kanal 2 er kraft (eller bytte om).

In [ ]:
plt.figure()
plt.plot(t, ch1)
plt.title('Kanal 1 vs Tid')
plt.xlabel('Tid [s]'); plt.ylabel('Spænding [V]')
plt.show()

plt.figure()
plt.plot(t, ch2)
plt.title('Kanal 2 vs Tid')
plt.xlabel('Tid [s]'); plt.ylabel('Spænding [V]')
plt.show()

def hf_var(x, w=50):
    kernel = np.ones(w) / w
    mov = np.convolve(x, kernel, mode='same')
    hf = x - mov
    return np.var(hf)

hf1, hf2 = hf_var(ch1), hf_var(ch2)
print('Højfrekvent varians — ch1:', hf1, ' ch2:', hf2)
# FLAG 3: CTF{EMG=?,FORCE=?}

## Udfordring 4 — Maksimal kraft
**Mål:** Find tid og værdi for **maksimal kraft**.

**FLAG 4:** `CTF{t_peak_force_s=XX.XXXX, peak_force_V=YY.YYYY}` (4 decimaler).

In [ ]:
# Antag at du identificerede ch_force i Udfordring 3
ch_force = ch2  # eller ch1, hvis omvendt
idx = np.argmax(ch_force)
t_peak = t[idx]
f_peak = ch_force[idx]
print('t_peak=', t_peak, ' f_peak=', f_peak)
# FLAG 4: CTF{t_peak_force_s=..., peak_force_V=...}

## Udfordring 5 — Kraft baseline
**Mål:** Beregn middel baseline-kraft over de første 0,5 s af målingen.

**FLAG 5:** `CTF{force_baseline_V=Z.ZZZZ}` (4 decimaler).

In [ ]:
t0 = t[0]
mask = (t >= t0) & (t < t0 + 0.5)
force_baseline = ch_force[mask].mean()
print('kraft-baseline [V]=', force_baseline)
# FLAG 5: CTF{force_baseline_V=...}

## Udfordring 6 — EMG-konvolut (moving RMS)
**Mål:** Beregn en EMG-konvolut via **fuldbølge-rektificering** og **moving RMS** vindue på 50 ms.

Trin:
1. Træk middelværdi fra EMG-kanalen.
2. Rektificér (`abs`).
3. Kvadrér, glidende middel (vindue `W = 0.05 * fs` samples), derefter `sqrt`.

**FLAG 6:** Tid og værdi for **maksimum af konvolutten**: `CTF{t_peak_env_s=AA.AAAA, env_peak=BB.BBBB}`.

In [ ]:
ch_emg = ch1  # eller ch2, hvis omvendt
W = int(0.05 * fs)
x = ch_emg - np.mean(ch_emg)
rect = np.abs(x)
kernel = np.ones(W) / W
env = np.sqrt(np.convolve(rect**2, kernel, mode='same'))
idx_env = np.argmax(env)
t_env_peak = t[idx_env]
env_peak = env[idx_env]
print('t_env_peak=', t_env_peak, ' env_peak=', env_peak)
# FLAG 6: CTF{t_peak_env_s=..., env_peak=...}

## Udfordring 7 — EMG–Kraft relation
**Mål:** Kvantificér hvor godt EMG-konvolutten følger kraften.

Trin:
- Nedprøvet begge signaler til 100 Hz ved simpel decimering (`int(fs/100)`).
- Beregn Pearson-korrelation.

**FLAG 7:** `CTF{corr_env_force=CC.CCCC}` (4 decimaler).

In [ ]:
step = int(fs // 100)
env_ds = env[::step]
force_ds = ch_force[::step]
m = min(env_ds.size, force_ds.size)
r = np.corrcoef(env_ds[:m], force_ds[:m])[0, 1]
print('corr(env, force)=', r)
# FLAG 7: CTF{corr_env_force=...}

## Udfordring 8 — Elektromekanisk delay (lag)
**Mål:** Estimér tidsforskydningen mellem EMG-konvolut og kraft via krydskorrelation ved 100 Hz.

Fortolkning: Hvis bedste korrelation ses ved **negativ** lag (konvolut fører), så går EMG-forandringer forud for kraften.

**FLAG 8:** `CTF{lag_seconds=DD.DD}` hvor negativ betyder EMG fører kraft.

In [ ]:
x2 = env_ds - np.mean(env_ds)
y2 = force_ds - np.mean(force_ds)
xc = np.correlate(x2, y2, mode='full')
lags = np.arange(-x2.size + 1, x2.size)
best = np.argmax(xc)
lag_samples = lags[best]
lag_seconds = lag_samples / 100.0
print('lag [s]=', lag_seconds)
# FLAG 8: CTF{lag_seconds=...}

## Udfordring 9 — Middel konvolut omkring peak-kraft
**Mål:** Beregn **middel EMG-konvolut** i et 0,5 s vindue centreret ved peak-kraft.

**FLAG 9:** `CTF{mean_env_peakwin=EE.EEEE}` (4 decimaler).

In [ ]:
t_start = t_peak - 0.25
t_end = t_peak + 0.25
mask2 = (t >= t_start) & (t <= t_end)
mean_env_peakwin = np.mean(env[mask2])
print('middel konvolut omkring peak-kraft =', mean_env_peakwin)
# FLAG 9: CTF{mean_env_peakwin=...}

## Udfordring 10 — Sanity checks
**Mål:** Rapporter total varighed og antal samples.

**FLAG 10:** `CTF{duration_s=FF.FFFF, N=NNNNN}`.

In [ ]:
duration = t[-1] - t[0]
N = t.size
print('varighed [s]=', duration, ' samples =', N)
# FLAG 10: CTF{duration_s=..., N=...}

## Udfordring 11 — Normalisér kraft med baseline
**Mål:** Baseline-korrigér kraftsignalet ved at **trække baseline fra**: `force_norm = ch_force - force_baseline`.

Rapportér den nye peakværdi og tidspunkt for peak efter baselinekorrektion.

**FLAG 11:** `CTF{t_peak_force_norm_s=GG.GGGG, peak_force_norm_V=HH.HHHH}`.

In [ ]:
force_norm = ch_force - force_baseline
idx_n = np.argmax(force_norm)
t_peak_norm = t[idx_n]
f_peak_norm = force_norm[idx_n]
print('t_peak_norm=', t_peak_norm, ' peak_force_norm=', f_peak_norm)
# FLAG 11: CTF{t_peak_force_norm_s=..., peak_force_norm_V=...}

## Udfordring 12 — EMG lineær konvolut via 1.-ordens IIR lavpas (10 Hz)
**Mål:** Implementér en **enpolet IIR lavpas** til at glatte den rektificerede EMG og dermed lave en lineær konvolut.

Trin:
1. Demean `ch_emg` og rektificér: `rect = abs(ch_emg - mean(ch_emg))`.
2. Vælg skæringsfrekvens `fc = 10 Hz`.
3. Sæt `alpha = exp(-2*pi*fc/fs)` og filtrér: `y[n] = (1-alpha)*rect[n] + alpha*y[n-1]` med `y[0] = rect[0]`.

Find tid og værdi for maksimum af denne IIR-konvolut.

**FLAG 12:** `CTF{iir_env_peak_t=II.IIII, iir_env_peak=JJ.JJJJ}`.

In [ ]:
fc = 10.0  # Hz
alpha = np.exp(-2.0 * np.pi * fc / fs)
rect2 = np.abs(ch_emg - np.mean(ch_emg))
iir_env = np.empty_like(rect2)
iir_env[0] = rect2[0]
for n in range(1, rect2.size):
    iir_env[n] = (1.0 - alpha) * rect2[n] + alpha * iir_env[n-1]
idx_iir = np.argmax(iir_env)
t_iir_peak = t[idx_iir]
iir_peak = iir_env[idx_iir]
print('t_iir_peak=', t_iir_peak, ' iir_peak=', iir_peak)
# FLAG 12: CTF{iir_env_peak_t=..., iir_env_peak=...}